# Homework 2: Learning about Datasets

In [1]:
#Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd

# NASA Earth Exchange (NEX) Downscaled Climate Projections (NEX-DCP30) dataset for the conterminous United States

## About the data
Modelled data generated from the General Circulation Model(GCM) runs under 33 different Coupled Model Intercomparison Project Phase 5(CMIP5) models. The modelled data projections are available for four greenhouse gas emissions scenarios known as Representative Concentration Pathways(RCPs)  labelled after the range of radiative forcing values in the year 2100 (2.6, 4.5, 6, and 8.5 W/m2).

## Data Access
Data can be download in a variety of ways at this link https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-dcp30 through the THREDDS Data Server.
For ease of use, our team used the NetcdfSubset method which allowed for input of desired consituents, geographic bounding box in latitude and longitude and time interval of interest. 

## Main constituents:
Precipitation (monthly mean of the daily precipitation rate)(kg/m2s)
Temperature (monthly mean of the daily predicted temperature)(deg K) 

## Data File Format: 
netcdf file
-one monthly averaged value for each month from 2006 to 2099
-monthly average can be based off of daily min, max, average and quartiles for each constituent
-your selected file will contain the variables you request for the spatial and temporal subset you specify in the NetcdfSubset access url 

## Metadata Format:
Since the data was in netcdf format, the metadata was contained within each downloaded file. The website source also provides a very useful Tech note which is accessed here: https://www.nccs.nasa.gov/sites/default/files/NEX-DCP30_Tech_Note.pdf





## 1. Write code to load in the dataset

## Loading Example Dataset 
Decided to focus on the RCP 8.5 dataset which is the most extreme emissions scenario for Santa Barbara area (9 cells centered on 34.4208° N, 119.6982° W). 

Downloaded Monthly Averages:
-Daily Mean Preciptation rate
-Daily Maximum Temperature
-Daily Minimum Temperature

In [2]:
# xarray was used to load in the data
sb_mean_precip = xr.open_dataset('sb_mean_precip_data.nc')
sb_mean_max_temp = xr.open_dataset('sb_mean_max_temp.nc')
sb_mean_min_temp = xr.open_dataset('sb_mean_min_temp.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/danielkerstan/Desktop/EDS_211/notebook/sb_mean_precip_data.nc'

In [ ]:
#view metadata
sb_mean_max_temp

## Date and Geospatial Format
Metadata indicates that date is in datetime64 format and geospatial information in stored as lat and lon(float64). 

# 2. Investigate data quality

## Spatial Coverage: 
Contiguous US at a spatial resolution of 30 arc-seconds/0.00833 degrees(approximately 800 meters)

## Temporal Coverage: 
Monthly average of 1950 through 2005 (“Retrospective Run”) and from 2006 to 2099 (“Prospective Run”)

The spatial and temporal coverage of this dataset are one of the key strength of the datsets. However, since it's modelled data, it comes with a set of assumptions. 

## Assumptions: 

Relative spatial patterns in temperature and precipitation observed from 1950 through 2005 will remain constant under future climate change. 

Does not add information beyond what is contained in the original CMIP5 scenarios, and preserves the frequency of periods of anomalously high and low temperature or precipitation (i.e., extreme events) within each individual CMIP5 scenario.

## Bias Correction-Spatial Disaggregation (BSCD)

***Problem: Biased data unsuitable for local level decision making.*** 

Original GCM model runs are run at coarse resolution which is not at the level of detail required for local decision making. In addition, although the projections are globally accurate there may be considerable bias at the local level since it does not take into account local topography.

***Solution: BSCD using PRISM (observational climate data from meteorological stations)***

The Bias-Correction step corrects the bias of the GCM data through comparisons performed against the observationally-based PRISM data.
The Spatial-Disaggregation step spatially interpolates the bias-corrected GCM data to the finer resolution grid of the 30-arc second PRISM data.

# 3. Consider appropriate use cases

The goal of this dataset is to “[Assist in] conducting studies of climate change impacts at local to regional scales, and to enhance public understanding of possible future climate patterns and climate impacts at the scale of individual neighborhoods and communities.”

This could be important for building and planning of cities, regions, or even individual neighborhoods.

Example use cases include: 

1. Watershed Management on a local level based on future precipitation trends

2. Building code updates based on projected changes in temperatures

3. Developing strategic plans for heatwaves, floods, wildfires and other natural disasters based on local level impacts 

The high spatial and temporal resolution allows for identification of detection of long-term trends on a local level. 

The monthly averaged data allows to identify potential increased risk to certain extreme events but not exact dates since that would not be feasible in modelling. 

Policymakers, think-tanks and business around the US regardless of their level can benefit from this dataset as they can limit their exposure to climate change risk based on these key indicators. 

Let’s take a closer look of our example for Santa Barbara:

In [ ]:
# store an object representing the time to use for the x-axis
sb_mean_precip_time = sb_mean_precip['time']

In [ ]:
# use groupby to get a single data value per time interval for the region of interest
sb_mean_precip_summary = sb_mean_precip.groupby('time').mean(['lat', 'lon'])

In [ ]:
# create a plot to show expected precipitation over the entire time period for the sb area
plt.figure(figsize = (10,6), dpi = 300)
plt.plot(sb_mean_precip_time, sb_mean_precip_summary['ens-avg_pr'])
plt.title("Monthly Average Precipitation")
plt.xlabel("Time")
plt.ylabel("Precipitation (kg m-2 s-1)")

In [ ]:
# establish a start date and end date to explore a more specific timeframe
start_date = np.datetime64('2021-10-31')
end_date = np.datetime64('2041-10-31')

# create a subset of the data that includes only this time period
sb_mean_precip_sub = sb_mean_precip["ens-avg_pr"].sel(time = slice(start_date, end_date))

In [ ]:
# store an object representing the time to use for the x-axis
sb_mean_precip_sub_time = sb_mean_precip_sub['time']

# use groupby to get a single data value per time interval for the region of interest
sb_mean_precip_sub_summary = sb_mean_precip_sub.groupby('time').mean(['lat', 'lon'])

In [ ]:
# plot this subset of the precipitation dataset
plt.figure(figsize = (10,6), dpi = 300)
plt.plot(sb_mean_precip_sub_time, sb_mean_precip_sub_summary)
plt.title("Monthly Average Precipitation")
plt.xlabel("Time")
plt.ylabel("Precipitation (kg m-2 s-1)")

In [ ]:
# repeat above steps to plot the maximum temperature data for the full time period
sb_mean_max_temp_time = sb_mean_max_temp['time']
sb_mean_max_temp_summary = sb_mean_max_temp.groupby('time').mean(['lat', 'lon'])
plt.figure(figsize = (10,6), dpi = 300)
plt.plot(sb_mean_max_temp_time, sb_mean_max_temp_summary['ens-avg_tasmax'])
plt.title("Monthly Average Maximum Near-Surface Air Temperature")
plt.xlabel("Time")
plt.ylabel("Temperature (Degrees Kelvin)")

In [ ]:
# repeat above steps to plot the minimum temperature data for the full time period
sb_mean_min_temp_time = sb_mean_min_temp['time']
sb_mean_min_temp_summary = sb_mean_min_temp.groupby('time').mean(['lat', 'lon'])
plt.figure(figsize = (10,6), dpi = 300)
plt.plot(sb_mean_min_temp_time, sb_mean_min_temp_summary['ens-avg_tasmin'])
plt.title("Monthly Average Minimum Near-Surface Air Temperature")
plt.xlabel("Time")
plt.ylabel("Temperature (Degrees Kelvin)")

# 4. Prepare dataset overview for class presentation

A short presentation was prepared for our class summarizing the results of the work above. The slides can be found here: https://docs.google.com/presentation/d/1gm9SebCXzmdPh4qMBduHCsOx_54i3758oSZKyAa5g7E/edit?usp=sharing

In [4]:
sb_county = gpd.read_file('tl_2019_06083_faces/tl_2019_06083_faces.shp')

xmin, ymin, xmax, ymax = sb_county.total_bounds

In [6]:
sb_county.total_bounds

array([-120.734382,   33.411024, -118.962728,   35.114678])